In [2]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
!pip install gmaps
import gmaps
import os

# Import API key
from api_keys import g_key

weather_py = "../output_data/cities.csv"
cities = pd.read_csv(weather_py)
cities.head()
gmaps.configure(api_key = g_key)

cities=cities.rename(columns = {"Max Temp":"MaxTemp", "Wind Speed":"WindSpeed"})
cities["Humidity"].max()


min_temp= 21.111
max_temp = 26.666
wind_speed = 10
cloudiness = 0

good_cities= cities.query(f"MaxTemp < {max_temp} and MaxTemp > {min_temp} and WindSpeed < {wind_speed} and Cloudiness == {cloudiness}")
good_cities.head()

hotel_df = good_cities
hotel_df['Hotel Name']=''
hotel_df.head()

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

locations["Lat"]= locations["Lat"].astype(str)
locations["Lng"]=locations["Lng"].astype(str)

hotels = pd.DataFrame({"City": hotel_df["City"],
                      "Country": hotel_df["Country"]})
hotels["Hotel Name"]=''
hotels["Lat"]=''
hotels["Lng"]=''

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights="Humidity", 
    dissipating=False,
    max_intensity = 100,
    point_radius = 3
)
fig.add_layer(heat_layer)
fig

<ipython-input-2-19846d7bce94>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''
<ipython-input-2-19846d7bce94>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations["Lat"]= locations["Lat"].astype(str)
<ipython-input-2-19846d7bce94>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

NameError: name 'weights' is not defined